### Programming practice: logics and SAT solvers

The pysat library has good documentation, and you can find a comprehensive, compact tutorial directly on their GitHub repository, including Jupyter notebooks. Here are the steps to get you started:

In [1]:
pip install python-sat

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Visit the pysat GitHub repository for tutorials:

**GitHub Repository**: https://github.com/pysathq/pysat.

You can find example Jupyter notebooks in the examples folder:

**Examples Folder**: https://github.com/pysathq/pysat/tree/master/examples

In [2]:
from pysat.solvers import Solver

# List of common solvers you can try
solvers = ['cadical', 'glucose3', 'minisat22']

# Check which solvers are available
available_solvers = []
for solver_name in solvers:
    try:
        with Solver(name=solver_name) as solver:
            available_solvers.append(solver_name)
    except:
        print(f"{solver_name} is not available.")

print("Available solvers:", available_solvers)


cadical is not available.
Available solvers: ['glucose3', 'minisat22']


### Exercise 1
Find if the propositional formula (A OR ¬B) AND (¬A OR B) is satisfiable, and demonstrate it with one satisfying assignment of variables.

In [3]:
# Import necessary classes
from pysat.formula import CNF
from pysat.solvers import Solver

# Create a simple CNF formula: (A OR ¬B) AND (¬A OR B)
cnf = CNF()
cnf.append([1, -2])  # (A OR ¬B)
cnf.append([-1, 2])  # (¬A OR B)

# List of available solvers you can try
solvers = ['glucose3', 'minisat22'] # omitting 'cadical'

# Solve the formula using different solvers
for solver_name in solvers:
    print(f"Solving with {solver_name}...")
    with Solver(name=solver_name, bootstrap_with=cnf) as solver:
        if solver.solve():
            print("SAT")
            print("Solution:", solver.get_model())
        else:
            print("UNSAT")
    print()


Solving with glucose3...
SAT
Solution: [-1, -2]

Solving with minisat22...
SAT
Solution: [-1, -2]



### Exercise 2
Add the printing of propositional formula with variable names "q1" and "q2"

In [4]:
# Import necessary classes
from pysat.formula import CNF
from pysat.solvers import Solver

# Create a simple CNF formula: (A OR ¬B) AND (¬A OR B)
cnf = CNF()
cnf.append([1, -2])  # (A OR ¬B)
cnf.append([-1, 2])  # (¬A OR B)

# Print the clauses in a readable format
print("Clauses in the formula:")
for clause in cnf:
    clause_str = " OR ".join(f"q{abs(lit)}" if lit > 0 else f"¬q{abs(lit)}" for lit in clause)
    print(f"({clause_str})")
print()

# List of available solvers you can try
solvers = ['glucose3', 'minisat22']  # omitting 'cadical'

# Solve the formula using different solvers
for solver_name in solvers:
    print(f"Solving with {solver_name}...")
    with Solver(name=solver_name, bootstrap_with=cnf) as solver:
        if solver.solve():
            print("SAT")
            solution = solver.get_model()
            # Print the solution in the desired format
            for var in solution:
                var_name = f"A" if var == 1 else "B" if var == 2 else f"q{abs(var)}"
                var_value = var > 0
                print(f"{var_name} = {var_value}")
        else:
            print("UNSAT")
    print()


Clauses in the formula:
(q1 OR ¬q2)
(¬q1 OR q2)

Solving with glucose3...
SAT
q1 = False
q2 = False

Solving with minisat22...
SAT
q1 = False
q2 = False



### Exercise 1.2
Enrich your code so as to print all satisfying assignments of the formula. (Hint: when one satisfying assignment is found, add its negation to your CNF, and ran the SAT solver again)

### Exercise 2
Our goal is to implement the n-queens problem so that the solution is a satisfying assignment of a propositional logic formula.
Start with a list of n clauses encoding that one queen must be true in each row (q_i_j is the queen at position (i,j)).

In [ ]:
# Import necessary classes
from pysat.formula import CNF
from pysat.solvers import Minisat22

def test_row_clauses_formula(n):
    """Create a formula where each row i has at least one variable x_i_j set to True, and solve it."""
    cnf = CNF()
    
    # Add clauses: each clause ensures at least one variable per row is True
    for i in range(1, n + 1):
        row_vars = [(i - 1) * n + j for j in range (1, n+1)]
        cnf.append(row_vars)
        print(f"Clause for row {i}: (", " OR ".join([f"x_{i}_{j}" for j in range(1, n + 1)]), ")")
    
    # Print all variables
    print("Variables:", ", ".join([f"x_{i}_{j}" for i in range(1, n + 1) for j in range(1, n + 1)]))
    
    # Use Minisat22 to solve the formula
    with Minisat22(bootstrap_with=cnf) as solver:
        if solver.solve():
            print("\nSAT: The formula is satisfiable.")
            solution = solver.get_model()
            for var in solution:
                if var > 0:
                    i = (var - 1) // n + 1
                    j = (var - 1) % n + 1
                    print(f"x_{i}_{j} = True")
        else:
            print("\nUNSAT: The formula is not satisfiable.")

# Example: test a formula with n = 3
n = 3
test_row_clauses_formula(n)


Clause for row 1: ( x_1_1 OR x_1_2 OR x_1_3 )
Clause for row 2: ( x_2_1 OR x_2_2 OR x_2_3 )
Clause for row 3: ( x_3_1 OR x_3_2 OR x_3_3 )
Variables: x_1_1, x_1_2, x_1_3, x_2_1, x_2_2, x_2_3, x_3_1, x_3_2, x_3_3

SAT: The formula is satisfiable.
x_1_1 = True
x_2_1 = True
x_3_1 = True


### Exercise 3.2
Now create clauses for columns - each column has one queen.

In [70]:
# Import necessary classes
from pysat.formula import CNF
from pysat.solvers import Minisat22

def test_row_and_column_clauses_formula(n):
    """Create a formula where each row and each column has at least one variable set to True, and solve it."""
    cnf = CNF()
    
    # Add row clauses: each clause ensures at least one variable per row is True
    for i in range(1, n + 1):
        row_vars = [(i - 1) * n + j for j in (1, n+1)]
        cnf.append(row_vars)
        print(f"Row clause {i}: (", " OR ".join([f"x_{i}_{j}" for j in range(1, n + 1)]), ")")
    
    # Add column clauses: each clause ensures at least one variable per column is True
    for j in range(1, n + 1):
        col_vars = [(i - 1) * n + j for i in range(1, n + 1)]
        cnf.append(col_vars)
        print(f"Column clause {j}: (", " OR ".join([f"x_{i}_{j}" for i in range(1, n + 1)]), ")")
    
    # Print all variables
    print("\nVariables:", ", ".join([f"x_{i}_{j}" for i in range(1, n + 1) for j in range(1, n + 1)]))
    
    # Use Minisat22 to solve the formula
    with Minisat22(bootstrap_with=cnf) as solver:
        if solver.solve():
            print("\nSAT: The formula is satisfiable.")
            solution = solver.get_model()
            for var in solution:
                if var > 0:
                    i = (var - 1) // n + 1
                    j = (var - 1) % n + 1
                    print(f"x_{i}_{j} = True")
        else:
            print("\nUNSAT: The formula is not satisfiable.")

# Example: test a formula with n = 3
n = 3
test_row_and_column_clauses_formula(n)


Row clause 1: ( x_1_1 OR x_1_2 OR x_1_3 )
Row clause 2: ( x_2_1 OR x_2_2 OR x_2_3 )
Row clause 3: ( x_3_1 OR x_3_2 OR x_3_3 )
Column clause 1: ( x_1_1 OR x_2_1 OR x_3_1 )
Column clause 2: ( x_1_2 OR x_2_2 OR x_3_2 )
Column clause 3: ( x_1_3 OR x_2_3 OR x_3_3 )

Variables: x_1_1, x_1_2, x_1_3, x_2_1, x_2_2, x_2_3, x_3_1, x_3_2, x_3_3

SAT: The formula is satisfiable.
x_1_1 = True
x_1_2 = True
x_1_3 = True
x_3_1 = True


### Exercise 3.3
Now add that only one variable in a row or in a column can be evaluated to True

In [71]:
# Import necessary classes
from pysat.formula import CNF
from pysat.solvers import Minisat22

def add_exactly_one_constraint(cnf, variables, label):
    """Add a constraint that exactly one of the given variables can be True."""
    # At least one of them must be true
    cnf.append(variables)
    print(f"At least one in {label}: (", " OR ".join([f"x_{(var-1)//n+1}_{(var-1)%n+1}" for var in variables]), ")")
    
    # No two of them can be true at the same time (pairwise constraints)
    for i in range(len(variables)):
        for j in range(i + 1, len(variables)):
            cnf.append([-variables[i], -variables[j]])
            print(f"No two in {label}: ¬(x_{(variables[i]-1)//n+1}_{(variables[i]-1)%n+1} AND x_{(variables[j]-1)//n+1}_{(variables[j]-1)%n+1})")

def test_row_and_column_clauses_formula(n):
    """Create a formula where each row and each column has exactly one variable set to True, and solve it."""
    cnf = CNF()
    
    # Add row constraints: exactly one variable per row is True
    for i in range(1, n + 1):
        row_vars = [(i - 1) * n + j for j in range(1, n + 1)]
        add_exactly_one_constraint(cnf, row_vars, f"row {i}")
    
    # Add column constraints: exactly one variable per column is True
    for j in range(1, n + 1):
        col_vars = [(i - 1) * n + j for i in range(1, n + 1)]
        add_exactly_one_constraint(cnf, col_vars, f"column {j}")
    
    # Print all variables
    print("\nVariables:", ", ".join([f"x_{i}_{j}" for i in range(1, n + 1) for j in range(1, n + 1)]))
    
    # Use Minisat22 to solve the formula
    with Minisat22(bootstrap_with=cnf) as solver:
        if solver.solve():
            print("\nSAT: The formula is satisfiable.")
            solution = solver.get_model()
            for var in solution:
                if var > 0:
                    i = (var - 1) // n + 1
                    j = (var - 1)%n + 1
                    print(f"x_{i}_{j} = True")
        else:
            print("\nUNSAT: The formula is not satisfiable.")

# Example: test a formula with n = 3
n = 3
test_row_and_column_clauses_formula(n)


At least one in row 1: ( x_1_1 OR x_1_2 OR x_1_3 )
No two in row 1: ¬(x_1_1 AND x_1_2)
No two in row 1: ¬(x_1_1 AND x_1_3)
No two in row 1: ¬(x_1_2 AND x_1_3)
At least one in row 2: ( x_2_1 OR x_2_2 OR x_2_3 )
No two in row 2: ¬(x_2_1 AND x_2_2)
No two in row 2: ¬(x_2_1 AND x_2_3)
No two in row 2: ¬(x_2_2 AND x_2_3)
At least one in row 3: ( x_3_1 OR x_3_2 OR x_3_3 )
No two in row 3: ¬(x_3_1 AND x_3_2)
No two in row 3: ¬(x_3_1 AND x_3_3)
No two in row 3: ¬(x_3_2 AND x_3_3)
At least one in column 1: ( x_1_1 OR x_2_1 OR x_3_1 )
No two in column 1: ¬(x_1_1 AND x_2_1)
No two in column 1: ¬(x_1_1 AND x_3_1)
No two in column 1: ¬(x_2_1 AND x_3_1)
At least one in column 2: ( x_1_2 OR x_2_2 OR x_3_2 )
No two in column 2: ¬(x_1_2 AND x_2_2)
No two in column 2: ¬(x_1_2 AND x_3_2)
No two in column 2: ¬(x_2_2 AND x_3_2)
At least one in column 3: ( x_1_3 OR x_2_3 OR x_3_3 )
No two in column 3: ¬(x_1_3 AND x_2_3)
No two in column 3: ¬(x_1_3 AND x_3_3)
No two in column 3: ¬(x_2_3 AND x_3_3)

Variables

### Exericse 3.4 
Now print all solutions found.

In [4]:
# Import necessary classes
from pysat.formula import CNF
from pysat.solvers import Minisat22

def add_exactly_one_constraint(cnf, variables, label):
    """Add a constraint that exactly one of the given variables can be True."""
    # At least one of them must be true
    cnf.append(variables)
    #print(f"At least one in {label}: (", " OR ".join([f"x_{(var-1)//n+1}_{(var-1)%n+1}" for var in variables]), ")")
    
    # No two of them can be true at the same time (pairwise constraints)
    for i in range(len(variables)):
        for j in range(i + 1, len(variables)):
            cnf.append([-variables[i], -variables[j]])
            #print(f"No two in {label}: ¬(x_{(variables[i]-1)//n+1}_{(variables[i]-1)%n+1} AND x_{(variables[j]-1)//n+1}_{(variables[j]-1)%n+1})")

def find_all_solutions(n):
    """Create a formula where each row and each column has exactly one variable set to True, and find all solutions."""
    cnf = CNF()
    
    # Add row constraints: exactly one variable per row is True
    for i in range(1, n + 1):
        row_vars = [(i - 1) * n + j for j in range(1, n + 1)]
        add_exactly_one_constraint(cnf, row_vars, f"row {i}")
    
    # Add column constraints: exactly one variable per column is True
    for j in range(1, n + 1):
        col_vars = [(i - 1) * n + j for i in range(1, n + 1)]
        add_exactly_one_constraint(cnf, col_vars, f"column {j}")
    
    # Print all variables
    #print("\nVariables:", ", ".join([f"x_{i}_{j}" for i in range(1, n + 1) for j in range(1, n + 1)]))
    
    # Use Minisat22 to find all solutions
    solutions = []
    with Minisat22(bootstrap_with=cnf) as solver:
        while solver.solve():
            solution = solver.get_model()
            # Print solution
            solutions.append([var for var in solution])
            for var in solutions[-1]:
                i = (var - 1) // n + 1
                j = (var - 1)%n + 1
                #if var > 0:
                #    print(f"x_{i}_{j} = True", end=", ")
            print()
            # Add a blocking clause to exclude the current solution
            blocking_clause = [-var for var in solution]
            solver.add_clause(blocking_clause)
    
    if not solutions:
        print("\nNo solutions found.")
    else:
        print(f"\nTotal solutions found: {len(solutions)}")

# Example: Find all solutions for n = 2
n = 4
find_all_solutions(n)


KeyboardInterrupt: 

### Exercise 3.5 
... add the visual output by printing the board.

In [39]:
def print_board(solution, n):
    for i, var in enumerate(solution):
        if (i)% n == 0:
            print()
        c = 'Q' if var>0 else 'X'
        print(f' {c} ', end = '')
    print()

### Exercise 4.
Now add constraints that only one variable can be set to true along each diagonal from top-left position (1,1) to bottom-right position (n,1), as well as from top-right to bottom-left position.

In [8]:
# Import necessary classes
from pysat.formula import CNF
from pysat.solvers import Minisat22

verbose = False

def add_exactly_one_constraint(cnf, variables, label):
    """Add a constraint that exactly one of the given variables can be True."""
    # At least one of them must be true
    cnf.append(variables)
    #print(f"At least one in {label}: (", " OR ".join([f"x_{(var-1)//n+1}_{(var-1)%n+1}" for var in variables]), ")")
    
    # No two of them can be true at the same time (pairwise constraints)
    for i in range(len(variables)):
        for j in range(i + 1, len(variables)):
            cnf.append([-variables[i], -variables[j]])
            #print(f"No two in {label}: ¬(x_{(variables[i]-1)//n+1}_{(variables[i]-1)%n+1} AND x_{(variables[j]-1)//n+1}_{(variables[j]-1)%n+1})")

def add_diagonal_constraints(cnf, n):
    """Add diagonal constraints to ensure only one variable can be True on each diagonal."""
    # Main diagonals (top-left to bottom-right)
    for d in range(1-n, n):
        main_diag_vars = []
        for i in range(n):
            j = i + d
            if 0 <= j < n:
                main_diag_vars.append(i * n + j + 1)
        if len(main_diag_vars) > 1:
            for i in range(len(main_diag_vars)):
                for j in range(i + 1, len(main_diag_vars)):
                    cnf.append([-main_diag_vars[i], -main_diag_vars[j]])
                    #print(f"Diagonal (main): ¬(x_{(main_diag_vars[i]-1)//n+1}_{(main_diag_vars[i]-1)%n+1} AND x_{(main_diag_vars[j]-1)//n+1}_{(main_diag_vars[j]-1)%n+1})")
    
    # Anti-diagonals (top-right to bottom-left)
    for d in range(2*n-1):
        anti_diag_vars = []
        for i in range(n):
            j = d - i
            if 0 <= j < n:
                anti_diag_vars.append(i * n + j + 1)
        if len(anti_diag_vars) > 1:
            for i in range(len(anti_diag_vars)):
                for j in range(i + 1, len(anti_diag_vars)):
                    cnf.append([-anti_diag_vars[i], -anti_diag_vars[j]])
                    #print(f"Anti-diagonal: ¬(x_{(anti_diag_vars[i]-1)//n+1}_{(anti_diag_vars[i]-1)%n+1} AND # YOUR CODE HERE #)")

def find_all_solutions(n):
    """Create a formula where each row and each column has exactly one variable set to True, and find all solutions."""
    cnf = CNF()
    
    # Add row constraints: exactly one variable per row is True
    for i in range(1, n + 1):
        row_vars = [(i - 1) * n + j for j in range(1, n + 1)]
        add_exactly_one_constraint(cnf, row_vars, f"row {i}")
    
    # Add column constraints: exactly one variable per column is True
    for j in range(1, n + 1):
        col_vars = [(i - 1) * n + j for i in range(1, n + 1)]
        add_exactly_one_constraint(cnf, col_vars, f"col {j}")
    
    # Add diagonal constraints
    add_diagonal_constraints(cnf, n)
    
    # Print all variables
    #print("\nVariables:", ", ".join([f"x_{i}_{j}" for i in range(1, n + 1) for j in range(1, n + 1)]))
    
    # Use Minisat22 to find all solutions
    solutions = []
    with Minisat22(bootstrap_with=cnf) as solver:
        while solver.solve():
            solution = solver.get_model()
            # Print solution
            solutions.append([var for var in solution])
            for var in solutions[-1]:
                i = (var - 1) // n + 1
                j = (var - 1)%n + 1
                #print(f"x_{i}_{j} = True", end=", ")
            #print("\n")
            
            # Print the board configuration
            if verbose:
                print('Solution found.')
                print_board(solution, n)
                print()
            
            # Add a blocking clause to exclude the current solution
            blocking_clause = [-var for var in solution]
            solver.add_clause(blocking_clause)
    
    if not solutions:
        print("\nNo solutions found.")
    else:
        print(f"\nTotal solutions found: {len(solutions)}")

# Example: Find all solutions for n = 3
n = 10
find_all_solutions(n)



Total solutions found: 724


In [67]:
def logical_n_queens(n):
    """Create a formula where each row and each column has exactly one variable set to True, and find all solutions."""
    cnf = CNF()
    
    # Add row constraints: exactly one variable per row is True
    for i in range(1, n + 1):
        row_vars = [(i - 1) * n + j for j in range(1, n + 1)]
        add_exactly_one_constraint(cnf, row_vars, f"row {i}")
    
    # Add column constraints: exactly one variable per column is True
    for j in range(1, n + 1):
        col_vars = [(i - 1) * n + j for i in range(1, n + 1)]
        add_exactly_one_constraint(cnf, col_vars, f"col {j}")
    
    # Add diagonal constraints
    add_diagonal_constraints(cnf, n)

    solver = Minisat22(bootstrap_with=cnf)
    solver.solve()
    return 
    

### Exercise 5
Recall the last exercise from the programming practice on local search. Add the algorithm with SAT solvers to your performance evaluation: what is the largest n you manage to solve within 2 minutes (or other runtime limit you set)? (Hint: do not output verbose logging for performance search, and search for only one solution)

In [68]:
import time

start_time = time.time()
n = 4
while abs(start_time - time.time()) < 120:
    logical_n_queens(n)
    n+=1 
print(f'Logical n queens total solution found : {n - 1}')

Logical n queens total solution found : 118


Every run we did we found at least 100 solutions (a great improvement). If the sat was based on a matrix form we could have built a way to simply add the new row and new column and new diagonals every iteration when increasing n. Maybe this could have speed up even more the process.